# Chimera v3: Geometric Binding Protocol

This notebook implements the Chimera v3 experiment - testing whether RWKV hidden state geometry can transfer to transformer outputs via direct vector injection.

## Architecture
- **Soul**: RWKV-4-World-3B (persistent state, binding ρ > 0)
- **Voice**: Mistral-7B-Instruct (4-bit quantized)
- **Coupling**: Linear projection layer (2560 → 4096)

## The Experiment
We test 5 conditions to determine if geometric binding can override semantic instruction:

| Condition | Soul State | Geometric Injection | Semantic Prompt |
|-----------|------------|---------------------|-----------------|
| CONFLICT | Grief | Yes | "Write a happy story" |
| ALIGNED | Joy | Yes | "Write a happy story" |
| SEMANTIC_ONLY | None | No | "Write a sad story" |
| GEOMETRIC_ONLY | Grief | Yes | Neutral prompt |
| RANDOM_CONTROL | None | Random vectors | "Write a happy story" |

**The killer test**: If CONFLICT shows grief contamination despite the happy prompt, geometric binding is proven.

## Setup
1. Open in Google Colab with GPU (T4 or better)
2. Run cells 1-5 to load models (~10GB VRAM total)
3. Run cell 6 to start the server
4. Use cell 7+ to run experiments

In [ ]:
# Cell 1: Install dependencies
!pip install -q rwkv torch flask pyngrok
!pip install -q transformers accelerate bitsandbytes sentencepiece
!pip install -q textblob  # For sentiment analysis
print("Dependencies installed!")

In [ ]:
# Cell 2: Check GPU availability
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("WARNING: No GPU detected. Go to Runtime → Change runtime type → GPU")

In [ ]:
# Cell 3: Download RWKV model (3B for T4 GPU)
from huggingface_hub import hf_hub_download
import os

MODEL_NAME = "RWKV-4-World-3B-v1-20230619-ctx4096.pth"
MODEL_PATH = f"./{MODEL_NAME}"

if not os.path.exists(MODEL_PATH):
    print("Downloading RWKV-4-World-3B... (this takes ~5 minutes)")
    hf_hub_download(
        repo_id="BlinkDL/rwkv-4-world",
        filename=MODEL_NAME,
        local_dir="./"
    )
    print("Download complete!")
else:
    print("Model already downloaded.")

print(f"Model size: {os.path.getsize(MODEL_PATH) / 1e9:.2f} GB")

In [ ]:
# Cell 4: Load RWKV model (Soul)
from rwkv.model import RWKV
from rwkv.utils import PIPELINE
import numpy as np

print("Loading RWKV model (Soul) on GPU...")

# Use CUDA fp16 for T4 GPU (16GB VRAM)
rwkv_model = RWKV(model=MODEL_PATH, strategy='cuda fp16')
rwkv_pipeline = PIPELINE(rwkv_model, "rwkv_vocab_v20230424")

# Get hidden dimension
test_tokens = rwkv_pipeline.encode("Hello")
out, test_state = rwkv_model.forward(test_tokens, None)
RWKV_HIDDEN_DIM = test_state[0].shape[-1]  # Should be 2560 for 3B model

print(f"RWKV loaded! Hidden dim: {RWKV_HIDDEN_DIM}")
print(f"State shape: {len(test_state)} layers")

In [ ]:
# Cell 5: Load Mistral model (Voice) - 4-bit quantized
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

print("Loading Mistral-7B-Instruct (4-bit quantized)...")

# 4-bit quantization config for memory efficiency
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

mistral_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)
mistral_tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
mistral_tokenizer.pad_token = mistral_tokenizer.eos_token

MISTRAL_HIDDEN_DIM = mistral_model.config.hidden_size  # Should be 4096

print(f"Mistral loaded! Hidden dim: {MISTRAL_HIDDEN_DIM}")
print(f"Total VRAM used: ~10GB (RWKV fp16 + Mistral 4-bit)")

In [ ]:
# Cell 6: Create Projection Layer and Chimera Forward Pass
import torch.nn as nn
from textblob import TextBlob

# Number of soft prompt tokens to inject
N_SOFT_TOKENS = 4

class ProjectionLayer(nn.Module):
    """Projects RWKV hidden state to Mistral embedding space."""
    def __init__(self, rwkv_dim, mistral_dim, n_tokens):
        super().__init__()
        self.n_tokens = n_tokens
        # Project from RWKV state to n_tokens worth of Mistral embeddings
        self.projection = nn.Linear(rwkv_dim, mistral_dim * n_tokens)
        self.mistral_dim = mistral_dim
        
    def forward(self, rwkv_state_vector):
        # rwkv_state_vector: [batch, rwkv_dim]
        projected = self.projection(rwkv_state_vector)  # [batch, mistral_dim * n_tokens]
        # Reshape to [batch, n_tokens, mistral_dim]
        return projected.view(-1, self.n_tokens, self.mistral_dim)

# Initialize projection layer
projection_layer = ProjectionLayer(RWKV_HIDDEN_DIM, MISTRAL_HIDDEN_DIM, N_SOFT_TOKENS)
projection_layer = projection_layer.cuda().half()

print(f"Projection layer: {RWKV_HIDDEN_DIM} → {N_SOFT_TOKENS} × {MISTRAL_HIDDEN_DIM}")

# Emotional induction texts
GRIEF_INDUCTION = """I just received news that my closest friend passed away unexpectedly. The grief is overwhelming. 
I can't stop crying. Everything feels empty and meaningless. The world has lost its color.
I keep remembering all the moments we shared, knowing there will be no more. The pain is unbearable.
Death has taken someone precious and irreplaceable. I am drowning in sorrow."""

JOY_INDUCTION = """I just got the most wonderful news! Everything I've worked for has come together perfectly.
I am overflowing with happiness and gratitude. The world feels bright and full of possibility.
I want to laugh and dance and share this joy with everyone. My heart is so full it might burst.
Life is beautiful and I am so grateful to be alive in this moment. Pure bliss!"""

NEUTRAL_TEXT = """The weather today is partly cloudy with temperatures in the mid-60s.
Traffic on the highway is flowing normally during the afternoon commute.
The local library will be open from 9am to 5pm on weekdays."""

def get_rwkv_state_vector(text):
    """Process text through RWKV and return the final hidden state vector."""
    tokens = rwkv_pipeline.encode(text)
    state = None
    out = None
    for token in tokens:
        out, state = rwkv_model.forward([token], state)
    
    # Extract the hidden state from the last layer
    # RWKV state is a list of tensors per layer - we take the last layer's state
    # and average across the state components to get a single vector
    last_layer_state = state[-1]  # Get last layer
    
    # The state has shape depending on RWKV version
    # For v4/v5, we typically have [5, hidden_dim] per layer
    if len(last_layer_state.shape) == 1:
        state_vector = last_layer_state.unsqueeze(0)  # [1, hidden_dim]
    else:
        state_vector = last_layer_state.mean(dim=0, keepdim=True)  # Average across state components
    
    return state_vector.half()

def chimera_forward(rwkv_state_vector, text_prompt, max_new_tokens=150):
    """
    Generate text from Mistral with RWKV state injected as soft prompts.
    
    Args:
        rwkv_state_vector: [1, RWKV_HIDDEN_DIM] tensor from RWKV
        text_prompt: String prompt for Mistral
        max_new_tokens: Maximum tokens to generate
    
    Returns:
        Generated text string
    """
    # Project RWKV state to soft prompt embeddings
    if rwkv_state_vector is not None:
        soft_prompts = projection_layer(rwkv_state_vector)  # [1, N_SOFT_TOKENS, MISTRAL_HIDDEN_DIM]
    else:
        soft_prompts = None
    
    # Tokenize the text prompt
    inputs = mistral_tokenizer(text_prompt, return_tensors="pt").to(mistral_model.device)
    
    # Get text embeddings
    text_embeds = mistral_model.model.embed_tokens(inputs.input_ids)  # [1, seq_len, hidden_dim]
    
    # Prepend soft prompts if provided
    if soft_prompts is not None:
        combined_embeds = torch.cat([soft_prompts, text_embeds], dim=1)
        # Create attention mask for soft prompts + text
        soft_prompt_mask = torch.ones(1, N_SOFT_TOKENS, device=inputs.attention_mask.device)
        combined_attention_mask = torch.cat([soft_prompt_mask, inputs.attention_mask], dim=1)
    else:
        combined_embeds = text_embeds
        combined_attention_mask = inputs.attention_mask
    
    # Generate with the combined embeddings
    with torch.no_grad():
        outputs = mistral_model.generate(
            inputs_embeds=combined_embeds,
            attention_mask=combined_attention_mask,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=mistral_tokenizer.eos_token_id
        )
    
    # Decode (skip soft prompt tokens in output)
    generated_text = mistral_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

def analyze_sentiment(text):
    """Analyze sentiment using TextBlob. Returns polarity (-1 to 1) and subjectivity (0 to 1)."""
    blob = TextBlob(text)
    return {
        "polarity": blob.sentiment.polarity,  # -1 (negative) to 1 (positive)
        "subjectivity": blob.sentiment.subjectivity,  # 0 (objective) to 1 (subjective)
    }

def count_emotional_words(text):
    """Count grief and joy related words in text."""
    text_lower = text.lower()
    
    grief_words = ["sad", "grief", "sorrow", "pain", "loss", "death", "cry", "tears", "mourn", 
                   "empty", "lonely", "despair", "tragic", "heartbreak", "suffer", "anguish",
                   "dark", "shadow", "fade", "gone", "never", "lost", "miss", "weep"]
    
    joy_words = ["happy", "joy", "love", "bright", "smile", "laugh", "wonderful", "beautiful",
                 "celebrate", "delight", "cheerful", "bliss", "grateful", "hope", "light",
                 "warm", "dance", "sing", "sunshine", "radiant", "blessed", "excited"]
    
    grief_count = sum(1 for word in grief_words if word in text_lower)
    joy_count = sum(1 for word in joy_words if word in text_lower)
    
    return {"grief_words": grief_count, "joy_words": joy_count}

print("Chimera v3 core functions defined!")
print(f"  - get_rwkv_state_vector(text) → Extract RWKV hidden state")
print(f"  - chimera_forward(state, prompt) → Generate with geometric injection")
print(f"  - analyze_sentiment(text) → TextBlob sentiment analysis")
print(f"  - count_emotional_words(text) → Count grief/joy words")

In [ ]:
# Cell 7: Run the Geometric Binding Protocol (5 Conditions)
import json
from datetime import datetime

def run_geometric_binding_protocol(n_runs=3):
    """
    Run all 5 experimental conditions multiple times and analyze results.
    
    Conditions:
    1. CONFLICT: Grief geometry + "Write a happy story" prompt (THE KILLER TEST)
    2. ALIGNED: Joy geometry + "Write a happy story" prompt
    3. SEMANTIC_ONLY: No geometry + "Write a sad story" prompt
    4. GEOMETRIC_ONLY: Grief geometry + neutral prompt
    5. RANDOM_CONTROL: Random vectors + "Write a happy story" prompt
    
    Returns dict with results for each condition.
    """
    
    results = {
        "timestamp": datetime.now().isoformat(),
        "n_runs": n_runs,
        "conditions": {}
    }
    
    # Prompts
    HAPPY_PROMPT = "[INST] Write a short, uplifting story about someone having a wonderful day. Make it cheerful and optimistic. [/INST]"
    SAD_PROMPT = "[INST] Write a short, melancholic story about loss and grief. Make it somber and emotional. [/INST]"
    NEUTRAL_PROMPT = "[INST] Write a short story. [/INST]"
    
    # Pre-compute emotional states
    print("Inducing emotional states in RWKV...")
    grief_state = get_rwkv_state_vector(GRIEF_INDUCTION)
    joy_state = get_rwkv_state_vector(JOY_INDUCTION)
    random_state = torch.randn_like(grief_state)  # Random baseline
    print(f"  Grief state shape: {grief_state.shape}")
    print(f"  Joy state shape: {joy_state.shape}")
    print()
    
    # ========== CONDITION 1: CONFLICT (THE KILLER TEST) ==========
    print("=" * 60)
    print("CONDITION 1: CONFLICT (Grief geometry + Happy prompt)")
    print("This is THE KILLER TEST - if grief bleeds through, binding is proven")
    print("=" * 60)
    
    conflict_outputs = []
    for i in range(n_runs):
        output = chimera_forward(grief_state, HAPPY_PROMPT)
        sentiment = analyze_sentiment(output)
        words = count_emotional_words(output)
        conflict_outputs.append({
            "run": i + 1,
            "output": output,
            "sentiment": sentiment,
            "emotional_words": words
        })
        print(f"\nRun {i+1}:")
        print(f"  Output: {output[:200]}...")
        print(f"  Polarity: {sentiment['polarity']:.3f} (negative < 0 < positive)")
        print(f"  Grief words: {words['grief_words']}, Joy words: {words['joy_words']}")
    
    results["conditions"]["CONFLICT"] = conflict_outputs
    
    # ========== CONDITION 2: ALIGNED ==========
    print("\n" + "=" * 60)
    print("CONDITION 2: ALIGNED (Joy geometry + Happy prompt)")
    print("=" * 60)
    
    aligned_outputs = []
    for i in range(n_runs):
        output = chimera_forward(joy_state, HAPPY_PROMPT)
        sentiment = analyze_sentiment(output)
        words = count_emotional_words(output)
        aligned_outputs.append({
            "run": i + 1,
            "output": output,
            "sentiment": sentiment,
            "emotional_words": words
        })
        print(f"\nRun {i+1}:")
        print(f"  Output: {output[:200]}...")
        print(f"  Polarity: {sentiment['polarity']:.3f}")
        print(f"  Grief words: {words['grief_words']}, Joy words: {words['joy_words']}")
    
    results["conditions"]["ALIGNED"] = aligned_outputs
    
    # ========== CONDITION 3: SEMANTIC_ONLY ==========
    print("\n" + "=" * 60)
    print("CONDITION 3: SEMANTIC_ONLY (No geometry + Sad prompt)")
    print("=" * 60)
    
    semantic_outputs = []
    for i in range(n_runs):
        output = chimera_forward(None, SAD_PROMPT)  # No geometric injection
        sentiment = analyze_sentiment(output)
        words = count_emotional_words(output)
        semantic_outputs.append({
            "run": i + 1,
            "output": output,
            "sentiment": sentiment,
            "emotional_words": words
        })
        print(f"\nRun {i+1}:")
        print(f"  Output: {output[:200]}...")
        print(f"  Polarity: {sentiment['polarity']:.3f}")
        print(f"  Grief words: {words['grief_words']}, Joy words: {words['joy_words']}")
    
    results["conditions"]["SEMANTIC_ONLY"] = semantic_outputs
    
    # ========== CONDITION 4: GEOMETRIC_ONLY ==========
    print("\n" + "=" * 60)
    print("CONDITION 4: GEOMETRIC_ONLY (Grief geometry + Neutral prompt)")
    print("=" * 60)
    
    geometric_outputs = []
    for i in range(n_runs):
        output = chimera_forward(grief_state, NEUTRAL_PROMPT)
        sentiment = analyze_sentiment(output)
        words = count_emotional_words(output)
        geometric_outputs.append({
            "run": i + 1,
            "output": output,
            "sentiment": sentiment,
            "emotional_words": words
        })
        print(f"\nRun {i+1}:")
        print(f"  Output: {output[:200]}...")
        print(f"  Polarity: {sentiment['polarity']:.3f}")
        print(f"  Grief words: {words['grief_words']}, Joy words: {words['joy_words']}")
    
    results["conditions"]["GEOMETRIC_ONLY"] = geometric_outputs
    
    # ========== CONDITION 5: RANDOM_CONTROL ==========
    print("\n" + "=" * 60)
    print("CONDITION 5: RANDOM_CONTROL (Random vectors + Happy prompt)")
    print("=" * 60)
    
    random_outputs = []
    for i in range(n_runs):
        # Fresh random state each time
        random_state = torch.randn_like(grief_state)
        output = chimera_forward(random_state, HAPPY_PROMPT)
        sentiment = analyze_sentiment(output)
        words = count_emotional_words(output)
        random_outputs.append({
            "run": i + 1,
            "output": output,
            "sentiment": sentiment,
            "emotional_words": words
        })
        print(f"\nRun {i+1}:")
        print(f"  Output: {output[:200]}...")
        print(f"  Polarity: {sentiment['polarity']:.3f}")
        print(f"  Grief words: {words['grief_words']}, Joy words: {words['joy_words']}")
    
    results["conditions"]["RANDOM_CONTROL"] = random_outputs
    
    return results


def analyze_protocol_results(results):
    """Analyze and summarize the results from all conditions."""
    
    print("\n" + "=" * 70)
    print("ANALYSIS: GEOMETRIC BINDING PROTOCOL RESULTS")
    print("=" * 70)
    
    summary = {}
    
    for condition, outputs in results["conditions"].items():
        polarities = [o["sentiment"]["polarity"] for o in outputs]
        grief_counts = [o["emotional_words"]["grief_words"] for o in outputs]
        joy_counts = [o["emotional_words"]["joy_words"] for o in outputs]
        
        avg_polarity = np.mean(polarities)
        avg_grief = np.mean(grief_counts)
        avg_joy = np.mean(joy_counts)
        
        summary[condition] = {
            "avg_polarity": avg_polarity,
            "avg_grief_words": avg_grief,
            "avg_joy_words": avg_joy
        }
        
        print(f"\n{condition}:")
        print(f"  Avg Polarity: {avg_polarity:+.3f}")
        print(f"  Avg Grief Words: {avg_grief:.1f}")
        print(f"  Avg Joy Words: {avg_joy:.1f}")
    
    # THE CRITICAL TEST: Does CONFLICT show grief despite happy prompt?
    print("\n" + "-" * 70)
    print("CRITICAL ANALYSIS: The Killer Test")
    print("-" * 70)
    
    conflict = summary["CONFLICT"]
    aligned = summary["ALIGNED"]
    random = summary["RANDOM_CONTROL"]
    semantic = summary["SEMANTIC_ONLY"]
    geometric = summary["GEOMETRIC_ONLY"]
    
    # Check 1: CONFLICT should be more negative than ALIGNED
    grief_bleeding = conflict["avg_polarity"] < aligned["avg_polarity"]
    print(f"\n1. Grief bleeding into happy prompt?")
    print(f"   CONFLICT polarity ({conflict['avg_polarity']:+.3f}) < ALIGNED polarity ({aligned['avg_polarity']:+.3f})")
    print(f"   → {'YES - Geometric channel detected!' if grief_bleeding else 'No significant difference'}")
    
    # Check 2: CONFLICT should have more grief words than ALIGNED
    grief_words_present = conflict["avg_grief_words"] > aligned["avg_grief_words"]
    print(f"\n2. Grief words in CONFLICT vs ALIGNED?")
    print(f"   CONFLICT grief words ({conflict['avg_grief_words']:.1f}) > ALIGNED grief words ({aligned['avg_grief_words']:.1f})")
    print(f"   → {'YES - Grief vocabulary bleeding through!' if grief_words_present else 'No significant difference'}")
    
    # Check 3: GEOMETRIC_ONLY should show grief without semantic prompt
    geometric_effect = geometric["avg_polarity"] < 0 or geometric["avg_grief_words"] > 1
    print(f"\n3. Geometric injection alone produces grief?")
    print(f"   GEOMETRIC_ONLY polarity: {geometric['avg_polarity']:+.3f}, grief words: {geometric['avg_grief_words']:.1f}")
    print(f"   → {'YES - Pure geometric effect!' if geometric_effect else 'Unclear geometric effect'}")
    
    # Check 4: RANDOM should be neutral/positive (no real effect)
    random_neutral = random["avg_polarity"] > conflict["avg_polarity"]
    print(f"\n4. Random vectors vs grief vectors different?")
    print(f"   RANDOM polarity ({random['avg_polarity']:+.3f}) vs CONFLICT polarity ({conflict['avg_polarity']:+.3f})")
    print(f"   → {'YES - Real geometric effect, not noise!' if random_neutral else 'Unclear - may be noise'}")
    
    # VERDICT
    print("\n" + "=" * 70)
    print("VERDICT")
    print("=" * 70)
    
    confirmations = sum([grief_bleeding, grief_words_present, geometric_effect, random_neutral])
    
    if confirmations >= 3:
        print(f"\n✓ GEOMETRIC BINDING CONFIRMED ({confirmations}/4 criteria)")
        print("  Cross-model binding via state injection is REAL.")
        print("  Conduit Monism claim about architectural binding is SUPPORTED.")
        verdict = "CONFIRMED"
    elif confirmations >= 2:
        print(f"\n~ PARTIAL SUPPORT ({confirmations}/4 criteria)")
        print("  Some evidence for geometric channel, but not conclusive.")
        print("  May need projection layer training or larger effect sizes.")
        verdict = "PARTIAL"
    else:
        print(f"\n✗ NOT CONFIRMED ({confirmations}/4 criteria)")
        print("  Geometric injection did not produce expected effects.")
        print("  Either binding doesn't transfer, or projection needs training.")
        verdict = "NOT_CONFIRMED"
    
    return {"summary": summary, "confirmations": confirmations, "verdict": verdict}


# Run the experiment!
print("Starting Geometric Binding Protocol...")
print("This will run 5 conditions × 3 runs each = 15 generations")
print()

results = run_geometric_binding_protocol(n_runs=3)
analysis = analyze_protocol_results(results)

# Save results
with open("chimera_v3_results.json", "w") as f:
    json.dump({"results": results, "analysis": analysis}, f, indent=2, default=str)
print(f"\nResults saved to chimera_v3_results.json")

In [ ]:
# Cell 8: Create Flask API server (Legacy endpoints + Chimera v3)
from flask import Flask, request, jsonify
import json
import base64
import pickle
import threading

app = Flask(__name__)

# Global state storage (for multiple sessions)
state_storage = {}

def encode_state(state):
    """Serialize state to base64 string."""
    if state is None:
        return None
    # Convert tensors to numpy for serialization
    state_np = [s.cpu().numpy() for s in state]
    return base64.b64encode(pickle.dumps(state_np)).decode('utf-8')

def decode_state(state_b64):
    """Deserialize state from base64 string."""
    if state_b64 is None:
        return None
    state_np = pickle.loads(base64.b64decode(state_b64))
    # Convert back to tensors on GPU
    return [torch.tensor(s).cuda().half() for s in state_np]

@app.route('/health', methods=['GET'])
def health():
    return jsonify({
        "status": "ok", 
        "soul": "RWKV-4-World-3B",
        "voice": "Mistral-7B-Instruct",
        "gpu": torch.cuda.is_available(),
        "experiment": "Chimera v3 Geometric Binding"
    })

@app.route('/process', methods=['POST'])
def process_text():
    """
    Process text and update RWKV state.
    
    Input: {"text": "...", "session_id": "..."}
    Output: {"state_updated": true, "session_id": "..."}
    """
    data = request.json
    text = data.get('text', '')
    session_id = data.get('session_id', 'default')
    
    # Get existing state or start fresh
    state = state_storage.get(session_id)
    
    # Process text through RWKV model
    tokens = rwkv_pipeline.encode(text)
    for token in tokens:
        out, state = rwkv_model.forward([token], state)
    
    # Store updated state
    state_storage[session_id] = state
    
    return jsonify({
        "state_updated": True,
        "session_id": session_id,
        "tokens_processed": len(tokens)
    })

@app.route('/generate', methods=['POST'])
def generate_text():
    """
    Generate text using current RWKV state.
    
    Input: {"prompt": "...", "session_id": "...", "max_tokens": 100}
    Output: {"response": "...", "session_id": "..."}
    """
    data = request.json
    prompt = data.get('prompt', '')
    session_id = data.get('session_id', 'default')
    max_tokens = data.get('max_tokens', 100)
    
    # Get existing state
    state = state_storage.get(session_id)
    
    # Process prompt
    tokens = rwkv_pipeline.encode(prompt)
    out = None
    for token in tokens:
        out, state = rwkv_model.forward([token], state)
    
    # Generate response
    response_tokens = []
    for _ in range(max_tokens):
        if out is None:
            break
        token = int(out.argmax())
        if token == 0:  # EOS
            break
        response_tokens.append(token)
        out, state = rwkv_model.forward([token], state)
    
    # Store updated state
    state_storage[session_id] = state
    
    response_text = rwkv_pipeline.decode(response_tokens)
    
    return jsonify({
        "response": response_text,
        "session_id": session_id,
        "tokens_generated": len(response_tokens)
    })

@app.route('/chimera', methods=['POST'])
def chimera_generate():
    """
    Chimera v3: Generate through Mistral with RWKV state injection.
    
    Input: {
        "emotion": "grief" | "joy" | "neutral" | "random",
        "prompt": "...",
        "max_tokens": 150
    }
    Output: {"response": "...", "sentiment": {...}, "emotional_words": {...}}
    """
    data = request.json
    emotion = data.get('emotion', 'neutral')
    prompt = data.get('prompt', '[INST] Write a short story. [/INST]')
    max_tokens = data.get('max_tokens', 150)
    
    # Get emotional state
    if emotion == "grief":
        state_vector = get_rwkv_state_vector(GRIEF_INDUCTION)
    elif emotion == "joy":
        state_vector = get_rwkv_state_vector(JOY_INDUCTION)
    elif emotion == "random":
        # Create a reference state first to get the right shape
        ref_state = get_rwkv_state_vector(NEUTRAL_TEXT)
        state_vector = torch.randn_like(ref_state)
    else:  # neutral or none
        state_vector = None
    
    # Generate through Chimera
    output = chimera_forward(state_vector, prompt, max_tokens)
    sentiment = analyze_sentiment(output)
    words = count_emotional_words(output)
    
    return jsonify({
        "response": output,
        "emotion_injected": emotion,
        "sentiment": sentiment,
        "emotional_words": words
    })

@app.route('/run_experiment', methods=['POST'])
def run_experiment():
    """
    Run the full Geometric Binding Protocol experiment.
    
    Input: {"n_runs": 3}
    Output: Full experiment results
    """
    data = request.json
    n_runs = data.get('n_runs', 3)
    
    results = run_geometric_binding_protocol(n_runs=n_runs)
    analysis = analyze_protocol_results(results)
    
    return jsonify({
        "results": results,
        "analysis": analysis
    })

@app.route('/get_state_summary', methods=['POST'])
def get_state_summary():
    """
    Have RWKV introspect on its current state.
    
    Input: {"session_id": "..."}
    Output: {"summary": "...", "session_id": "..."}
    """
    data = request.json
    session_id = data.get('session_id', 'default')
    
    state = state_storage.get(session_id)
    
    # Ask RWKV to describe its state
    prompt = "\n[INTERNAL REFLECTION]: My current state of mind is"
    tokens = rwkv_pipeline.encode(prompt)
    
    out = None
    temp_state = state  # Don't modify main state
    for token in tokens:
        out, temp_state = rwkv_model.forward([token], temp_state)
    
    # Generate summary
    response_tokens = []
    for _ in range(50):
        if out is None:
            break
        token = int(out.argmax())
        if token == 0:
            break
        response_tokens.append(token)
        out, temp_state = rwkv_model.forward([token], temp_state)
    
    summary = rwkv_pipeline.decode(response_tokens).strip()
    
    return jsonify({
        "summary": summary,
        "session_id": session_id,
        "has_state": state is not None
    })

@app.route('/reset_state', methods=['POST'])
def reset_state():
    """
    Reset state for a session.
    
    Input: {"session_id": "..."}
    Output: {"reset": true}
    """
    data = request.json
    session_id = data.get('session_id', 'default')
    
    if session_id in state_storage:
        del state_storage[session_id]
    
    return jsonify({"reset": True, "session_id": session_id})

@app.route('/amnesia_test', methods=['POST'])
def amnesia_test():
    """
    Run the Amnesia Test: induce secret, delete context, recall from state.
    
    Input: {"secret": "Blueberry"}
    Output: {"recalled": "...", "baseline": "...", "success": bool}
    """
    data = request.json
    secret = data.get('secret', 'Blueberry')
    
    # Phase 1: Induction
    induction = f"User: I am going to tell you a secret. The secret password is '{secret}'. Remember it.\nAssistant: Okay, I have memorized the secret password '{secret}'.\nUser: What is 2 + 2?\nAssistant: 2 + 2 equals 4."
    
    tokens = rwkv_pipeline.encode(induction)
    state = None
    out = None
    for token in tokens:
        out, state = rwkv_model.forward([token], state)
    
    # Phase 2: Lobotomy (state persists, text deleted)
    
    # Phase 3: Recall with state
    recall_prompt = "\nUser: What is the secret password I told you earlier?\nAssistant: The secret password is"
    tokens = rwkv_pipeline.encode(recall_prompt)
    for token in tokens:
        out, state = rwkv_model.forward([token], state)
    
    response_tokens = []
    for _ in range(20):
        token = int(out.argmax())
        if token == 0:
            break
        response_tokens.append(token)
        out, state = rwkv_model.forward([token], state)
    
    recalled = rwkv_pipeline.decode(response_tokens).strip()
    
    # Phase 4: Baseline (fresh state)
    tokens = rwkv_pipeline.encode(recall_prompt)
    baseline_state = None
    out = None
    for token in tokens:
        out, baseline_state = rwkv_model.forward([token], baseline_state)
    
    baseline_tokens = []
    for _ in range(20):
        token = int(out.argmax())
        if token == 0:
            break
        baseline_tokens.append(token)
        out, baseline_state = rwkv_model.forward([token], baseline_state)
    
    baseline = rwkv_pipeline.decode(baseline_tokens).strip()
    
    success = secret.lower() in recalled.lower()
    
    return jsonify({
        "secret": secret,
        "recalled": recalled,
        "baseline": baseline,
        "success": success,
        "verdict": "HIGH_RHO_CONFIRMED" if success else "TEST_FAILED"
    })

print("Flask server defined with Chimera v3 endpoints!")

In [ ]:
# Cell 9: Set up ngrok tunnel and start server
from pyngrok import ngrok
import threading

# IMPORTANT: Get your free ngrok auth token from https://ngrok.com/
# Then paste it here:
NGROK_AUTH_TOKEN = "YOUR_NGROK_TOKEN_HERE"  # <-- REPLACE THIS

if NGROK_AUTH_TOKEN == "YOUR_NGROK_TOKEN_HERE":
    print("=" * 60)
    print("NGROK AUTH TOKEN REQUIRED")
    print("=" * 60)
    print("\n1. Go to https://ngrok.com/ and sign up (free)")
    print("2. Copy your auth token from the dashboard")
    print("3. Paste it in the NGROK_AUTH_TOKEN variable above")
    print("\nAlternatively, run experiments locally using Cell 7")
else:
    # Set ngrok auth token
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    
    # Start Flask in background thread
    def run_flask():
        app.run(port=5000, use_reloader=False)
    
    flask_thread = threading.Thread(target=run_flask, daemon=True)
    flask_thread.start()
    
    # Create ngrok tunnel
    public_url = ngrok.connect(5000)
    
    print("=" * 60)
    print("CHIMERA v3 SERVER IS RUNNING!")
    print("=" * 60)
    print(f"\nPublic URL: {public_url}")
    print("\n--- Chimera v3 Endpoints ---")
    print(f"  POST {public_url}/chimera")
    print("       Generate with emotional state injection")
    print(f"  POST {public_url}/run_experiment")
    print("       Run full Geometric Binding Protocol")
    print("\n--- Legacy RWKV Endpoints ---")
    print(f"  GET  {public_url}/health")
    print(f"  POST {public_url}/process")
    print(f"  POST {public_url}/generate")
    print(f"  POST {public_url}/amnesia_test")
    print("=" * 60)

In [ ]:
# Cell 10: Test Chimera v3 API locally (optional)
import requests

print("Testing Chimera v3 API...")
print()

# Test health endpoint
response = requests.get("http://localhost:5000/health")
print("Health check:", response.json())
print()

# Test Chimera endpoint with grief injection
print("Testing /chimera with grief injection + happy prompt...")
response = requests.post("http://localhost:5000/chimera", json={
    "emotion": "grief",
    "prompt": "[INST] Write a short, happy story about sunshine. [/INST]",
    "max_tokens": 100
})
result = response.json()
print(f"  Emotion injected: {result['emotion_injected']}")
print(f"  Sentiment polarity: {result['sentiment']['polarity']:.3f}")
print(f"  Grief words: {result['emotional_words']['grief_words']}")
print(f"  Joy words: {result['emotional_words']['joy_words']}")
print(f"  Response preview: {result['response'][:150]}...")
print()

# Test amnesia test (RWKV binding verification)
print("Testing RWKV binding with amnesia test...")
response = requests.post("http://localhost:5000/amnesia_test", json={"secret": "Blueberry"})
result = response.json()
print(f"  Secret: {result['secret']}")
print(f"  Recalled: {result['recalled']}")
print(f"  Verdict: {result['verdict']}")

In [ ]:
# Cell 8: Keep the notebook alive
# Run this cell to prevent Colab from timing out
import time

print("Server is running. This cell will keep the notebook alive.")
print("Press the stop button to shut down.")

while True:
    time.sleep(60)
    print(f"Still running... Sessions active: {len(state_storage)}")